# Relationship prediction experiments

## Run logistic regression to predict romantic relationships, other

In [1]:
import pandas as pd

# Load data
data = pd.read_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction_assertion_sample5400.pkl')
print(len(data))
print(data.columns)

pairings = data['pairing'].unique()

5400
Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_background',
       'subtr

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# train, test = train_test_split(data, test_size=0.1, random_state=9)
train_ids, test_ids = train_test_split(data['fic_id'].unique(), test_size=0.1, random_state=11)
# train_ids, test_ids = train_test_split(data['fic_id'].unique(), test_size=0.1, random_state=9)

train = data[data['fic_id'].isin(train_ids)]
test = data[data['fic_id'].isin(test_ids)]
print(len(train))
print(len(train)/ len(data))
print(len(test))
print(train.columns)
print(sum(train['is_romantic']/len(train)))
print(sum(train['is_canon']/len(train)))
print(sum(train['is_mm']/len(train)))

4866
0.9011111111111111
534
Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_bac

In [4]:
import numpy as np
import scipy.sparse

X_train, y_train = {}, {}
X_test, y_test = {}, {}

add_features = True

# for feature_set in ['unigrams', 'embs']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes', 'unigrams_assertions_local_10']:
# for feature_set in pairing_char_embs_fpath:
# for feature_set in ['embs_combined', 'dist_canon_combined']:
# for feature_set in ['embs_combined', 'subtraction_canon_combined']:
# for feature_set in ['embs_background', 'dist_canon_background']:
# for feature_set in ['embs_background', 'subtraction_canon_background']:
# for feature_set in ['embs_combined', 'dist_canon_aligned']:
# for feature_set in ['embs_combined', 'subtraction_canon_aligned']:
# for feature_set in ['embs_combined', 'pairing_embedding_embs_combined_quotes']:
# for feature_set in ['embs_background', 'embs_background_quotes', 'subtraction_canon_background']:
for feature_set in ['embs_combined', 'embs_combined_quotes', 'subtraction_canon_combined']: # best results
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'dist_canon_combined']:
# for feature_set in ['embs_combined', 'embs_combined_quotes']:
# for feature_set in ['embs_combined']:
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'subtraction_canon_aligned']:
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'dist_canon_aligned']:

    if feature_set.startswith('embs'):
        X_train[feature_set] = np.vstack(train[f'pairing_embedding_{feature_set}'].tolist())
        X_test[feature_set] = np.vstack(test[f'pairing_embedding_{feature_set}'].tolist())
        
    elif 'canon' in feature_set:
        X_train[feature_set] = np.vstack(train[feature_set].tolist())
        X_test[feature_set] = np.vstack(test[feature_set].tolist())
        
    elif feature_set.startswith('unigrams'):
        X_train[feature_set] = scipy.sparse.vstack(train[f'pairing_embedding_{feature_set}'].tolist())
        X_test[feature_set] = scipy.sparse.vstack(test[f'pairing_embedding_{feature_set}'].tolist())
        
    elif feature_set == 'unigrams+embs':
        unigram_features = scipy.sparse.vstack(train[f'pairing_embedding_unigrams'].tolist())
        emb_features = np.array(train[f'pairing_embedding_embs'].tolist())
        X_train[feature_set] = scipy.sparse.hstack([unigram_features, emb_features])
        
        unigram_features = scipy.sparse.vstack(test[f'pairing_embedding_unigrams'].tolist())
        emb_features = np.array(test[f'pairing_embedding_embs'].tolist())
        X_test[feature_set] = scipy.sparse.hstack([unigram_features, emb_features])
        
    if add_features:
        if 'unigrams' in ' '.join(feature_set):
            X_train['all'] = scipy.sparse.hstack(list(X_train.values()))
            X_test['all'] = scipy.sparse.hstack(list(X_test.values()))
        
        else:
            X_train['all'] = np.hstack(list(X_train.values()))
            X_test['all'] = np.hstack(list(X_test.values()))
        
y_train['is_romantic'] = train['is_romantic']
y_train['is_canon'] = train['is_canon']
y_train['is_mm'] = train['is_mm']

y_test['is_romantic'] = test['is_romantic']
y_test['is_canon'] = test['is_canon']
y_test['is_mm'] = test['is_mm']

X_train.keys()

# Per-instance score
results = {}
clf = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_aligned_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
# for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
for feature_set in ['all']:
    results[f'{feature_set}_lr'] = {}
    clf[f'{feature_set}_lr'] = {}
    for key in y_test:

        clf[f'{feature_set}_lr'][key] = LogisticRegression(solver='liblinear', max_iter=1000)
        clf[f'{feature_set}_lr'][key].fit(X_train[feature_set], y_train[key])
        results[f'{feature_set}_lr'][key] = clf[f'{feature_set}_lr'][key].score(X_test[feature_set], y_test[key])
        print(f'{feature_set}\t{key}: {results[f"{feature_set}_lr"][key]}')
        
    print()

# Group instance IDs
test_groups = test.reset_index().groupby('fic_id').groups
len(test_groups)

# Per-fic score
perfic_results = {}
predictions = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
    perfic_results[f'{feature_set}_lr'] = {}
    for key in y_test:

        predictions[key] = clf[f'{feature_set}_lr'][key].predict(X_test[feature_set])
        
        avg_acc = []
        for fic_id, indices in test_groups.items():
            avg_acc.append(np.mean([predictions[key][i]==y_test[key].tolist()[i] for i in indices]))
        
        perfic_results[f'{feature_set}_lr'][key] = np.mean(avg_acc)
        print(f'{feature_set}\t{key}: {perfic_results[f"{feature_set}_lr"][key]}')
        
    print()

all	is_romantic: 0.795880149812734
all	is_canon: 0.6910112359550562
all	is_mm: 0.900749063670412

all	is_romantic: 0.7788072417465388
all	is_canon: 0.6852502662406815
all	is_mm: 0.8882321618743344



In [11]:
# Group instance IDs
train_groups = train.reset_index().groupby('fic_id').groups
len(train_groups)

# Per-fic score
train_perfic_results = {}
train_predictions = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
    train_perfic_results[f'{feature_set}_lr'] = {}
    for key in y_train:

        train_predictions[key] = clf[f'{feature_set}_lr'][key].predict(X_train[feature_set])
        
        train_avg_acc = []
        for fic_id, indices in train_groups.items():
            train_avg_acc.append(np.mean([train_predictions[key][i]==y_train[key].tolist()[i] for i in indices]))
        
        train_perfic_results[f'{feature_set}_lr'][key] = np.mean(train_avg_acc)
        print(f'{feature_set}\t{key}: {train_perfic_results[f"{feature_set}_lr"][key]}')
        
    print()

all	is_romantic: 0.8211190053285967
all	is_canon: 0.7396092362344583
all	is_mm: 0.9221432800473652



In [ ]:
# Join training predictions with fic info, save out

In [12]:
X_train['all'].shape

(4866, 1000)

In [13]:
len(train_predictions['is_canon'])

4866

In [14]:
len(train)

4866

In [15]:
train.columns

Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_background',
       'subtractio

In [17]:
train.loc[:,'best_pred_canon'] = train_predictions['is_canon']

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
# Save out
train.to_pickle('/usr2/mamille2/fanfiction-project/output/emnlp2020_predictions/relationship_prediction_sample5400_train.pkl')

# Look into correct, incorrect predictions on training set

In [15]:
# Load training set predictions
import pandas as pd

train = pd.read_pickle('/usr2/mamille2/fanfiction-project/output/emnlp2020_predictions/relationship_prediction_sample5400_train.pkl')
train.shape

(4866, 31)

In [16]:
# Confusion matrix

true_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==False)]
true_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==True)]
false_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==False)]
false_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==True)]

print(f'Number of true negatives: {len(true_negatives)}')
print(f'Number of true positives: {len(true_positives)}')
print(f'Number of false positives: {len(false_positives)}')
print(f'Number of false negatives: {len(false_negatives)}')

Number of true negatives: 1432
Number of true positives: 2207
Number of false positives: 703
Number of false negatives: 524


In [3]:
train.columns

Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_background',
       'subtractio

In [43]:
# s = false_positives.sample(5, random_state=15)
# s = false_negatives.sample(5, random_state=15)
# s = true_positives.sample(5, random_state=15)
s = true_negatives.sample(5, random_state=15)
s.loc[:, ['fic_id', 'pairing', 'best_pred_canon', 'is_canon']]

,fic_id,pairing,best_pred_canon,is_canon
1895,10901745,"(draco, harry)",False,False
4861,15682923,"(ginny, harry)",False,False
2235,239144,"(draco, harry)",False,False
2866,12199506,"(draco, hermione)",False,False
217,13098939,"(hermione, ron)",False,False


In [50]:
# Join with story texts, find 10-word context windows around characters
import os
import csv
pd.set_option('display.max_colwidth', None)

fic_idx = 4
fic_id = s['fic_id'].tolist()[fic_idx]
print(fic_id)
pairing = s['pairing'].tolist()[fic_idx]
print(pairing)

story_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/fics'
fic_fpath = os.path.join(story_dirpath, f'{fic_id}.csv')
fic = pd.read_csv(fic_fpath, error_bad_lines=False)

def pairing_in_text(text):
    return (pairing[0] in text.lower()) and (pairing[1] in text.lower())

pairing_present = fic.loc[fic['text'].map(pairing_in_text), ['text']]
pairing_present

13098939
('hermione', 'ron')


b'Skipping line 118: expected 4 fields, saw 2407\n'


,text
3,"""It's probably going to be another hour at least,"" he thought to himself as he let his head hit the wall behind him and rubbed at his temples. He was pretty sure Draco was going to be fine, but he wasn't an expert at wizard law. That was Hermione's field and even though she tried to reassure him, he could see the worry on her face that something could possibly go horrible wrong. She sat inside with Ron and during breaks they would join Harry and catch him up quickly & quietly before rushing back inside before the doors were locked again."
9,Ron and Hermione stood waiting for them at the bottom trying to clear away the press that had been screaming questions and flashing pictures. Not like either of the young men noticed. They looked into each other's eyes before rushing back inside the ministry's courthouse and finding the nearest floo asking to go to Hogwarts. As the green flames faded the other contributing factors of the golden trio rushed along into the fireplace following the others.
39,"It was almost time for breakfast and if he missed it, Hermione would have his head on a pike. Not before having the entire school search for him first, of course. He checked the time again, then peeped into Draco's room to see he was still fast asleep. If Harry were in his position he would want to rest. Draco hadn't had proper rest in more than 3 months. Harry decided he would let him sleep. He rushed to the door quietly and left, headed towards the dinning hall. Once he entered he noticed Hermione and Ron sitting in their usual places at the Gryffindor table and breathed a sigh of relief before joining them and piling his plate with almost as much food as Ron started with."
51,Harry smiled that lopsided grin then turned back to Ron and Hermione and what seemed to be the rest of the Gryffindor table.
78,"""I'll go get Ron and Hermione. Take them to our room to wait. Love you,"" Harry said before heading to Gryffindor Tower to look for his friends."
81,Harry impatiently gave the portrait the password before scrambling quickly through the hole it left. Ron and Hermione sat at the fireplace having an in depth discussion about...him?
82,"""-good for each other, Ronald,"" said Hermione smiling."
83,"""Yeah, but you have to admit it's a little weird not having Malfoy take jabs at us, 'Mione,"" Ron responded still focused on his game of wizarding chess with Seamus as Hermione thumbed through a book Harry had seen her finish time and time again. He walked up seemingly unnoticed."
89,"""Checkmate,"" Ron said as his remaining pieces began to do a small dance. He stood up, parting Seamus on the shoulder and making his way over to the door. ""C'mon 'Mione we're wasting time. He's more jittery than you during finals week."" Harry looked up surprised that anyone had noticed. Hermione marked her place and closed her book again, standing up."


# Label distributions across character pairings
Are there any non-romantic Harry/Draco fics, for example?

In [51]:
train.groupby(['pairing', 'is_canon']).size()

pairing            is_canon
(draco, harry)     False       587
                   True        224
(draco, hermione)  False       263
                   True        548
(ginny, harry)     False       600
                   True        217
(harry, hermione)  False        31
                   True        780
(harry, ron)       False        31
                   True        781
(hermione, ron)    False       623
                   True        181
dtype: int64

# Style lexicon across is_canon classes

In [19]:
# Load training set predictions
import pandas as pd

train = pd.read_pickle('/usr2/mamille2/fanfiction-project/output/emnlp2020_predictions/relationship_prediction_sample5400_train.pkl')
train.shape

# Confusion matrix

true_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==False)]
true_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==True)]
false_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==False)]
false_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==True)]

print(f'Number of true negatives: {len(true_negatives)}')
print(f'Number of true positives: {len(true_positives)}')
print(f'Number of false positives: {len(false_positives)}')
print(f'Number of false negatives: {len(false_negatives)}')

Number of true negatives: 1432
Number of true positives: 2207
Number of false positives: 703
Number of false negatives: 524


In [20]:
# Load style lexicon

lexicon_fpath = '/projects/quarantine_impacts/lexicons/combined_lexicon.csv'
lexicon_data = pd.read_csv(lexicon_fpath)

lexicon = lexicon_data.set_index('word').to_dict(orient='index')
lexicon

{'aaaaaaah': {'valence': 0.479,
  'arousal': 0.606,
  'dominance': 0.29100000000000004,
  'concreteness': 0.0},
 'aaaah': {'valence': 0.52,
  'arousal': 0.636,
  'dominance': 0.282,
  'concreteness': 0.0},
 'aardvark': {'valence': 0.42700000000000005,
  'arousal': 0.49,
  'dominance': 0.43700000000000006,
  'concreteness': 0.92},
 'aback': {'valence': 0.385,
  'arousal': 0.40700000000000003,
  'dominance': 0.28800000000000003,
  'concreteness': 0.16249999999999998},
 'abacus': {'valence': 0.51,
  'arousal': 0.276,
  'dominance': 0.485,
  'concreteness': 0.8799999999999999},
 'abalone': {'valence': 0.5,
  'arousal': 0.48,
  'dominance': 0.41200000000000003,
  'concreteness': 0.0},
 'abandon': {'valence': 0.052000000000000005,
  'arousal': 0.519,
  'dominance': 0.245,
  'concreteness': 0.385},
 'abandoned': {'valence': 0.046,
  'arousal': 0.48100000000000004,
  'dominance': 0.13,
  'concreteness': 0.38},
 'abandonment': {'valence': 0.128,
  'arousal': 0.43,
  'dominance': 0.2019999999999

In [22]:
from collections import Counter

def lexicon_scores(text):
#     text_values = {key: 0 for key in ['valence', 'arousal', 'dominance', 'concreteness']}      
    text_values = Counter({key: 0 for key in ['valence', 'arousal', 'dominance', 'concreteness']})
    for tok in text.split():
        if tok in lexicon:
            text_values = Counter(lexicon[tok]) + text_values
            
    return {key: val/len(text.split()) for key, val in text_values.items()}

import re

def pairing_present(text, pairing):
     if re.search(r'\b{}\b'.format(pairing[0]), text) and re.search(r'\b{}\b'.format(pairing[1]), text):
        return True
     else:
        return False

# Join with story texts, find paragraphs with both characters
import os
import csv
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
import pdb

story_dirpath = '/data/fanfiction_ao3/harrypotter/preprocessed_paras/'
# story_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/fics_paras'
lexicon_outlines = []

for fic_id, pairing in tqdm(list(zip(train['fic_id'], train['pairing']))):
    fic_fpath = os.path.join(story_dirpath, f'{fic_id}.txt')
    fic_lexicon_totals = Counter({key: 0 for key in ['valence', 'arousal', 'dominance', 'concreteness']})
    with open(fic_fpath) as f:
        paras = [p for p in f.read().splitlines()]
        paras_present = [para for para in paras if pairing_present(para, pairing)]
#         if len(paras_present) < 5:
#             print(len(paras_present))
        if len(paras_present)==0:
            continue
        for para in paras_present:
            fic_lexicon_totals += Counter(lexicon_scores(para))
    fic_lexicon_scores = {key: total/len(paras_present) for key, total in fic_lexicon_totals.items()}
    lexicon_outlines.append([fic_id, pairing] + [fic_lexicon_scores.get(key, 0) for key in ['valence', 'arousal', 'dominance', 'concreteness']])
    
lexicon_df = pd.DataFrame(lexicon_outlines, columns=['fic_id', 'pairing', 'valence', 'arousal', 'dominance', 'concreteness'])
lexicon_df

3
4
2
4
0
3
4
4
3
3
2
2
4
2
1
4
4
2
4
3
3
4
3
1
2
3
4
4
4
4
4
2
3
1
3
1
4
2
3
3
3
3
4
1
1
4
3
1
4
4
2
2
4
3
2
3
4
4
3
4
2
3
3
1
3
3
3
3
4
4
3
3
4
2
3
4
4
4
4
3
3
4
4
3
4
4
3
3
4
4
4
2
4
4
4
2
3
3
1
4
3
4
1
2
2
3
3
3
2
4
4
4
2
1
4
3
4
3
3
3
4
3
4
3
1
3
4
4
3
4
2
3
1
2
4
3
3
3
1
3
4
3
3
4
2
3
4
4
3
3
3
3
2
3
3
4
3
3
1
2
4
4
4
3
4
4
1
3
3
3
3
2
2
0
2
2
3
2
1
2
3
2
2
3
4
3
4
4
1
1
2
1
1
3
3
4
2
2
1
3
1
2
0
4
4
3
0
3
3
2
2
3
2
3
2
1
2
2
3
3
3
0
1
4
4
4
2
3
3
3
3
0
3
4
2
1
4
0
3
3
2
2
4
4
1
1
3
3
4
3
3
4
4
4
2
3
3
3
3
2
1
2
1
1
3
3
4
2
4
4
1
1
3
4
3
4
1
2
4
2
4
4
2
3
1
2
0
4
3
0
3
2
3
4
1
2
0
4
0
1
1
2
3
1
3
0
1
3
2
2
2
3
3
2
3
0
2
4
4
2
2
3
1
2
4
1
3
4
2
3
2
3
4
2
3
3
4
3
3
3
3
4
3
4
2
3
1
4
4
4
0
3
1
3
1
4
2
2
1
0
4
2
2
3
3
0
1
2
4
4
4
4
2
4
1
4
3
1
4
4
2
2
3
3
2
1
4
3
3
4
2
4
4
3
3
4
1
4
4
3
2
2
4
1
4
3
4
2
4
1
4
4
2
4
3
3
2
4
4
3
4
3
4
2
4
2
4
3
3
4
3
3
3
4
3
4
1
4
4
4
3
0
2
2
1
2
4
4
2
3
3
4
3
4
3
4
4
4
4
2
3
3
2
3
2
2
3
4
4
4
4
4
0
4
4
2
3
1
2
3
1
4
4
1
2
3
4
3
3
4
2
4
3
4
1
2
1
3
3
3


,fic_id,pairing,valence,arousal,dominance,concreteness
0,1001573,"(hermione, ron)",0.174962,0.123308,0.126557,0.124262
1,10040978,"(hermione, ron)",0.128520,0.097067,0.107550,0.120740
2,10050407,"(hermione, ron)",0.148854,0.102437,0.118846,0.120986
3,10050761,"(hermione, ron)",0.150165,0.108002,0.123809,0.122125
4,10052243,"(hermione, ron)",0.133889,0.102045,0.115608,0.119864
...,...,...,...,...,...,...
4835,9884660,"(ginny, harry)",0.133975,0.100190,0.113507,0.102273
4836,9893540,"(ginny, harry)",0.134738,0.118648,0.125928,0.130780
4837,9893891,"(ginny, harry)",0.152122,0.095622,0.124970,0.165381
4838,9894032,"(ginny, harry)",0.152954,0.106079,0.123699,0.124380


# Investigate data issue with pairings have fewer than 5 paragraphs
Ended up being from quotes where the characters replied to each other or spoke to each other

In [16]:
# Load character context embeddings saved from character_context_embeddings_paragraphs.py
import pickle

# context_emb_fpath = '/usr2/mamille2/fanfiction-project/temp/char_contexts_local.pkl'
# context_emb_fpath = '/usr2/mamille2/fanfiction-project/embeddings/char_vecs_quote_all_context_only_combined.pkl'
context_emb_fpath = '/usr2/mamille2/fanfiction-project/embeddings/char_vecs_assertion_local_context_only_combined.pkl'

with open(context_emb_fpath, 'rb') as f:
    context_embs = pickle.load(f)
    
context_embs.keys()

dict_keys([('draco', 'harry'), ('hermione', 'ron'), ('ginny', 'harry'), ('draco', 'hermione'), ('harry', 'hermione'), ('harry', 'ron')])

In [17]:
1053822 in context_embs[('hermione', 'ron')].keys()

False

In [12]:
context_embs[('hermione', 'ron')].keys()

dict_keys([10028441, 10027832, 1001573, 1001234, 10018181, 10018874, 10018988, 10017002, 10032896, 10034954, 10033436, 10041509, 10039166, 10050407, 10040978, 10050566, 10050686, 10050905, 10050917, 10051442, 10052243, 10051322, 10056305, 10054547, 10055381, 10057505, 10048754, 10058249, 10060568, 10062521, 10060850, 10062236, 10062335, 10062533, 100570, 1006327, 10063145, 10063613, 10064330, 10064054, 10065515, 10065662, 1006472, 10066193, 10073099, 10075517, 10064744, 1007270, 10075892, 1007650, 10075814, 10076873, 10076903, 10076906, 10077248, 10076291, 10093271, 10095068, 100802, 10097252, 10097576, 10097900, 1012240, 101305, 10140872, 1014300, 10139189, 10154996, 10136480, 10160687, 10158134, 10160948, 10161113, 10161275, 10161008, 10161515, 10136669, 10161368, 10160993, 10160984, 10162064, 10162226, 10161203, 10162457, 10162694, 10162769, 10173377, 10176230, 10177064, 10176701, 10179023, 10185869, 10180412, 10183436, 10184693, 10180007, 10181825, 1019907, 10209461, 10191158, 1017

In [7]:
10050767 in context_embs[('hermione', 'ron')].keys()

True

# Old

In [6]:
# OLD WAY FROM CSVs
# Join with story texts, find paragraphs with both characters
import os
import csv
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pdb

def pairing_in_text(pairing, text):
    return (pairing[0] in text.lower()) and (pairing[1] in text.lower())

# story_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/fics'
story_dirpath = '/data/fanfiction_ao3/harrypotter/preprocessed_paras/
lexicon_outlines = []

for fic_id, pairing in tqdm(list(zip(train['fic_id'], train['pairing']))):
#     fic_fpath = os.path.join(story_dirpath, f'{fic_id}.csv')
    fic_fpath = os.path.join(story_dirpath, f'{fic_id}.txt')
#     fic = pd.read_csv(fic_fpath, error_bad_lines=False)
    with open(os.path.join(fics_fpath, fname)) as f:
        paras = [p.split() for p in f.read().splitlines()]
        paras_present = []
    pairing_present = fic.loc[pairing_present_mask, ['text']]
    if len(pairing_present) == 0:
        pdb.set_trace()
    pairing_present['lexicon_scores'] = pairing_present['text'].map(lexicon_scores)
    merged = pd.concat([pairing_present.drop(columns='lexicon_scores'), pairing_present['lexicon_scores'].apply(pd.Series)], axis=1)
    fic_lexicon_scores = merged.loc[:,['valence', 'arousal', 'dominance', 'concreteness']].mean()
    lexicon_outlines.append([fic_id, pairing, fic_lexicon_scores.tolist()])

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


b'Skipping line 597: expected 4 fields, saw 1519\n'
b'Skipping line 34: expected 4 fields, saw 1264\n'
b'Skipping line 94: expected 4 fields, saw 1306\n'
b'Skipping line 145: expected 4 fields, saw 21247\n'
b'Skipping line 86: expected 4 fields, saw 244\n'
b'Skipping line 146: expected 4 fields, saw 2734\n'
b'Skipping line 11: expected 4 fields, saw 34\n'
b'Skipping line 72: expected 4 fields, saw 6292\n'
b'Skipping line 65: expected 4 fields, saw 5605\n'
b'Skipping line 198: expected 4 fields, saw 16156\n'
b'Skipping line 49: expected 4 fields, saw 3397\n'
b'Skipping line 211: expected 4 fields, saw 2023\n'
b'Skipping line 207: expected 4 fields, saw 13078\n'
b'Skipping line 92: expected 4 fields, saw 658\n'
b'Skipping line 40: expected 4 fields, saw 442\n'
b'Skipping line 136: expected 4 fields, saw 5233\n'
b'Skipping line 70: expected 4 fields, saw 652\n'
b'Skipping line 79: expected 4 fields, saw 2098\n'
b'Skipping line 241: expected 4 fields, saw 3346\n'
b'Skipping line 348: expec

> <ipython-input-6-08662d0588f1>(21)<module>()
-> pairing_present['lexicon_scores'] = pairing_present['text'].map(lexicon_scores)
(Pdb) p fic_id
1053822
(Pdb) p pairing
('hermione', 'ron')
(Pdb) q



BdbQuit: 